In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `instr-finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-auth

In [2]:
!pip install trl==0.11.4 --quiet
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [4]:
wiki_data = load_dataset("wikipedia", language="en", date="20220301", split="train[:2500]")

wiki_data = wiki_data.train_test_split(test_size=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Step 1 : Model Pre-training using Causal Language Modeling

## 1.1 Load Wiki-dataset for Model Pre-training

In [4]:
# Loading Mistral-7B-v0.1 to pre-train

base_model_id = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 1.2 Tokenize Wiki Dataset

In [6]:
# Tokenize wiki Data
def tokenize(sentences):
    return tokenizer(sentences['text'], max_length = 512, truncation = True, padding='max_length', return_tensors ='pt', add_special_tokens = True)

tokenized_wiki_data = wiki_data.map(tokenize, batched = True, remove_columns=['id', 'url', 'title', 'text'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## 1.3 Define Model for pre-training

### - Reducing Mistral-7B size for easier and faster training

In [7]:
from transformers import MistralForCausalLM, MistralConfig

max_length = 512

config = MistralConfig(
    hidden_size=768,
    sliding_window=768,
    intermediate_size=3072,
    max_position_embeddings=max_length,
    num_attention_heads=32,
    num_hidden_layers=8,)

model = MistralForCausalLM(config)

## 1.4 Define Data Collator for Model Pre-training

In [8]:
#Training using Causal Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

args = TrainingArguments(output_dir="mistral-pretraining",push_to_hub=True)

#Training function on wiki_train & wiki_test dataset
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_wiki_data["train"],
    eval_dataset=tokenized_wiki_data["test"],)

trainer.train()

<ipython-input-8-97dd8598e180>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mithilwork01 (mithilwork01-dreamlineai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,6.769100


TrainOutput(global_step=750, training_loss=6.50011962890625, metrics={'train_runtime': 662.0462, 'train_samples_per_second': 9.063, 'train_steps_per_second': 1.133, 'total_flos': 1714335252480000.0, 'train_loss': 6.50011962890625, 'epoch': 3.0})

In [9]:
#push the model to hugging-face hub
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Mithilhf01/mistral-pretraining/commit/40707101de8003c9b8581784ad6395b452dfa775', commit_message='End of training', commit_description='', oid='40707101de8003c9b8581784ad6395b452dfa775', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mithilhf01/mistral-pretraining', endpoint='https://huggingface.co', repo_type='model', repo_id='Mithilhf01/mistral-pretraining'), pr_revision=None, pr_num=None)

#Step 2: Supervised Finetuning training on Stanford's Alpaca dataset

## 2.1 Load Dataset for supervised fine-tuning

In [10]:
#loading stanford alpaca dataset
dataset = load_dataset("tatsu-lab/alpaca", split="train[:2000]")

## 2.2 Load the pre-trained model

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = 'Mithilhf01/mistral-pretraining'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

## 2.3 Defining Data collator for SFT -
### - Next token prediction(only for Responses. In other words, tokens corresponding to question will not be used while computing loss function).

In [12]:
from trl import DataCollatorForCompletionOnlyLM

response_template = "# Response:"
response_template_ids = tokenizer.encode(
     response_template,
     add_special_tokens=False)

data_collator = DataCollatorForCompletionOnlyLM(
     response_template_ids,
     tokenizer=tokenizer)

## 2.4 Defining Trainer function for SFT

In [13]:
from trl import SFTTrainer, SFTConfig

args = TrainingArguments(
    output_dir="mistral-supervised", push_to_hub=True,)

trainer = SFTTrainer(
    model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=dataset,
    data_collator=data_collator,
    dataset_text_field="text")

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:156: UserWarning: Could not find response key `[422, 12107, 28747]` in the following instance: <s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide a rap verse including the given words.

### Input:
words: 

Step,Training Loss
500,0.000000


Streaming output truncated to the last 5000 lines.
### Instruction:
Translate the following phrase into French.

### Input:
I miss you

### Response:
Je te manque. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:156: UserWarning: Could not find response key `[422, 12107, 28747]` in the following instance: <s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Provide a plan to get good sleep each night.

### Response:
Getting good sleep each night is essential for your health, well-being, and productivity. To ensure good sleep each night, it is important to have a consistent sleeping schedule and set a routine that can help you create a comfortable environment for sleep. It is also important to avoid consuming caffeine late in the day and limit your exposure to scr

TrainOutput(global_step=750, training_loss=0.0, metrics={'train_runtime': 274.8428, 'train_samples_per_second': 21.831, 'train_steps_per_second': 2.729, 'total_flos': 729521080565760.0, 'train_loss': 0.0, 'epoch': 3.0})

In [14]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Mithilhf01/mistral-supervised/commit/2238b453a62ddb7faa1803fb531dd78ab2bed631', commit_message='End of training', commit_description='', oid='2238b453a62ddb7faa1803fb531dd78ab2bed631', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mithilhf01/mistral-supervised', endpoint='https://huggingface.co', repo_type='model', repo_id='Mithilhf01/mistral-supervised'), pr_revision=None, pr_num=None)

# Step 3 : Fine-tuning using RLHF.
###  - RLHF with PPO optimization on  Anthropics-HH RLHF dataset with text classification model as reward model.

## 3.1 Load Anthropic/hh-rlhf data for RLHF Training

In [36]:
rlhf_dataset = load_dataset("Anthropic/hh-rlhf", split='train[:2000]')

README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

test.jsonl.gz:   0%|          | 0.00/875k [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

## 3.2 Preprocess and tokenize data

In [46]:
from collections import defaultdict
import torch
def preprocess_function(sentences):
    new_sentences = {
        "input_ids_chosen": list(), "attention_mask_chosen": list(),
        "input_ids_rejected": list(), "attention_mask_rejected": list(),}

    for chosen, rejected in zip(sentences["chosen"], sentences["rejected"]):
        tokenized_chosen = tokenizer(chosen, padding=True, truncation = True, max_length = 512, return_tensors ='pt', add_special_tokens = True)
        tokenized_rejected = tokenizer(rejected, padding=True, truncation = True, max_length = 512, return_tensors ='pt', add_special_tokens = True)

        new_sentences["input_ids_chosen"].append(tokenized_chosen["input_ids"].squeeze())
        new_sentences["attention_mask_chosen"].append(tokenized_chosen["attention_mask"].squeeze())
        new_sentences["input_ids_rejected"].append(tokenized_rejected["input_ids"].squeeze())
        new_sentences["attention_mask_rejected"].append(tokenized_rejected["attention_mask"].squeeze())

    return new_sentences

tokenizer.pad_token = tokenizer.eos_token
rlhf_tokenized_data = rlhf_dataset.map(preprocess_function, batched=True,  remove_columns=rlhf_dataset.column_names)
rlhf_tokenized_data.set_format(type="torch")

## 3.3 Reward model

### - Using SFT fine-tuned model and replacing LM head with classification head. Idea is to use classication output as Reward score.

In [44]:
from transformers import AutoModelForSequenceClassification

model_id = 'Mithilhf01/mistral-supervised'

reward_model = AutoModelForSequenceClassification.from_pretrained(model_id)
reward_model.config.pad_token_id = tokenizer.pad_token_id
reward_model.config.eos_token_id = tokenizer.eos_token_id
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Mithilhf01/mistral-supervised and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3.4 Defining Reward Trainer

In [47]:
from trl import RewardTrainer, RewardConfig
from transformers import TrainingArguments

#Define Training args
training_args = TrainingArguments(
    output_dir="mistral-reward",
    num_train_epochs=4,
    push_to_hub=True,
    report_to="none",
    learning_rate=1e-5,
)

#Training Reward Model on tokenized rlhf data
trainer = RewardTrainer(model=reward_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=rlhf_tokenized_data,)

trainer.train()

/usr/local/lib/python3.11/dist-packages/trl/trainer/reward_trainer.py:120: FutureWarning: Using `transformers.TrainingArguments` for `args` is deprecated and will be removed in a future version. Please use `RewardConfig` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/reward_trainer.py:174: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/reward_trainer.py:199: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/reward_trainer.py:208: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `RewardTrainer.__init__`. Use `processing

TypeError: RewardTrainer.compute_loss() got an unexpected keyword argument 'num_items_in_batch'

In [ ]:
trainer.push_to_hub()

## 3.5 Load data for PPO training

### - Use last 2000 samples from dataset used for SFT to train RLHF-PPO model

In [5]:
# Loading last 2000 data samples using during SFT for RLHF PP0 training
ppo_dataset = load_dataset("tatsu-lab/alpaca", split="train[-2000:]")

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

## 3.6 Define PPO Model

### - Adding a value head to our SFT Fine-tuned model to use it for PPO training.

In [6]:
from trl import AutoModelForCausalLMWithValueHead

# Get SFT Fine-tuned model
model_id = 'Mithilhf01/mistral-supervised'

ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

## 3.7 Tokenize PPO dataset

In [7]:
# Tokenize Dataset
def tokenize(sentences):
    sentences["input_ids"] = tokenizer.encode(sentences["instruction"], )
    return sentences

ppo_tokenized_dataset = ppo_dataset.map(tokenize, batched=False)
ppo_tokenized_dataset .set_format(type="torch")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# Define custome collator to convert list of dicts to dict to lists(needed for PPO Trainer)

def ppo_collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

## 3.8 Define PPO Trainer with correct configurations

### - Use Reference model as same PP0 model. Define Reward pipeline to score sentences using reward model.

In [9]:
from trl import PPOConfig, PPOTrainer, create_reference_model
from transformers import pipeline

# Define PPO Config
config = PPOConfig(
   remove_unused_columns=False)

# Using Reference Model same as PPO_model
ref_model = create_reference_model(ppo_model)

# Define reward model using pipeline to use our reward model
reward_pipeline = pipeline(model='Mithilhf01/mistral-reward')

ppo_trainer = PPOTrainer(
   config,
   ppo_model,
   ref_model,
   tokenizer,
   dataset=ppo_tokenized_dataset,
   data_collator=ppo_collator)

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/154M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


## 3.9 Helper function to format input to reward model

In [10]:
def format_reward_input(instruction, response):
    return 'Human: {} \n\n Assistant: {}'.format(instruction, response)

## 3.10 Train RLHF based PPO model

In [11]:
import torch
epochs = 3

gen_kwargs = {"min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,}

for epoch in range(epochs):
    for batch in ppo_trainer.dataloader:
        query_tensors = batch["input_ids"]

        # Get response from SFTModel
        response_tensors = ppo_trainer.generate(
            query_tensors,
            return_prompt=False,
            **gen_kwargs)

        batch["response"] = [
            tokenizer.decode(r.squeeze()) for r in response_tensors ]

        # Data ready to input reward model
        texts = [
            format_reward_input(q, r) for q, r in zip(batch["instruction"], batch["response"])]

        # Get reward score
        outputs = reward_pipeline(texts)
        rewards = [torch.tensor(output["score"]) for output in outputs]

        # Run PPO step
        ppo_trainer.step(
            query_tensors,
            response_tensors,
            rewards)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
ppo_trainer.push_to_hub('mistral-ppo')

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mithilhf01/mistral-ppo/commit/69749dee7f2e73a358ab2af3d6edcab40921e4f1', commit_message='Push model using huggingface_hub.', commit_description='', oid='69749dee7f2e73a358ab2af3d6edcab40921e4f1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mithilhf01/mistral-ppo', endpoint='https://huggingface.co', repo_type='model', repo_id='Mithilhf01/mistral-ppo'), pr_revision=None, pr_num=None)